# Summarized Metrics
This sheet is going to calculate the various metrics averaged over
- a sequnce (for sequences with multiple jumps)
- a frog (averaging over all sequences)
- the entire data set
This will be re-done with the old jump definition, and then done from scratch with the new jump definition

## What metrics are being calculated?
This was previously done in the Indv Jmp Analysis Notebook
- average max velocity | min and max max velocity
    - total + Vx, Vy
- average propulsion time | min and max propulsion time
- avg Vx for entire jump
- avg Vx below the water
    - comparing the 2 jump definitions needs some thought here
- avg Vx above water
- avg wait time | min and max wait time (between landing and next jump prep)
- avg height | min and max height
- avg range | min and max range
- avg height below water (before jump? i.e. starting depth?) | min and max
- avg range below water (after jump - glide distance?)
    - I need to see if distance moved between prep and water exit signifigant
- avg body angle below water | min and max
    - comparing the 2 jump definitions needs some thought here
- avg body angle above water | min and max
- avg body angle at water exit/launch | min and max
- avg body angle at water land | min and max
- avg body angle at max velocity | min and max
- avg TRAJECTORY angle at max velocity | min and max
    - if I do Vx and Vy, this is redundant
- avg height/depth at propulsion end | min and max
- avg height/depth at propulsion start | min and max
    - this would def be launch depth
- avg height/depth at prep start | min and max
    - hopefully would be comparable to propulsion start

In [4]:
from vidHandlr import *
import numpy as np
import cv2, time
import pickle as pik
import pandas as pd
import ezodf, csv, re, os
from alldata_20131031 import *
import matplotlib.pyplot as plt

In [5]:
import re

In [6]:
import pyperclip as clipbd

In [7]:
pd.options.display.max_columns = None

In [8]:
from alldata_20200526 import * #need frog size in cm

In [9]:
import statsmodels.api as sm
from statsmodels.stats.weightstats import DescrStatsW #for getting std, etc for weighted

## Lets tackle the new jump definition first

In [15]:
datasheet = '../AllData_20200526_strictjmpdef.xlsx'

In [16]:
datasheet

'../AllData_20200526_strictjmpdef.xlsx'

In [18]:
sheet = pd.read_excel(datasheet, sheet_name='StrictJmpDef')

In [19]:
sheet.columns

Index(['Frog', 'Sequence', 'ID', 'Jmp #', 'fps', 'px/SVL', 'Frame water exit',
       'Frame water land', 'Unnamed: 8', 'Actual height (px)',
       'Actual range (px)', 'fit height (px)', 'fit range (px)', 'Unnamed: 13',
       'Body Angle @ WC launch', 'Body Angle @ WC land', 'Unnamed: 16',
       'Idx of Max Velocity', 'Frame of Max Velocity',
       'Vx @ Max Velocity (px/sec)', 'Vy @ Max Velocity (px/sec)',
       'V @ Max Velocity (px/sec)', 'V angle (trajectory angle) at Max V',
       'Body Angle @ Max Velocity', 'Unnamed: 24', 'Unnamed: 25',
       'Frame start propulsion', 'Frame End propulsion',
       'Propulsion Time (frames)', 'Unnamed: 29', 'Frame start prep (HL)',
       'Frame end prep (HL)', 'Duration HL prep (frames)', 'Unnamed: 33',
       'Wait time (btw prev land and prep start)', 'Unnamed: 35',
       'Avg hor Velocity above water (px/sec)',
       'Avg hor Velocity below water (px/sec) – BEFORE jump',
       'Avg hor Velocity below water (px/sec) – AFTER jump',


In [20]:
# remove unnamed columsn:
sheet = sheet.loc[:, ~sheet.columns.str.contains('^Unnamed')]

In [21]:
#drop the spaces between frogs for easy readability
sheet= sheet[sheet['Frog'].notna()]

In [22]:
sheet

,Frog,Sequence,ID,Jmp #,fps,px/SVL,Frame water exit,Frame water land,Actual height (px),Actual range (px),fit height (px),fit range (px),Body Angle @ WC launch,Body Angle @ WC land,Idx of Max Velocity,Frame of Max Velocity,Vx @ Max Velocity (px/sec),Vy @ Max Velocity (px/sec),V @ Max Velocity (px/sec),V angle (trajectory angle) at Max V,Body Angle @ Max Velocity,Frame start propulsion,Frame End propulsion,Propulsion Time (frames),Frame start prep (HL),Frame end prep (HL),Duration HL prep (frames),Wait time (btw prev land and prep start),Avg hor Velocity above water (px/sec),Avg hor Velocity below water (px/sec) – BEFORE jump,Avg hor Velocity below water (px/sec) – AFTER jump,Avg hor vel total (launch land launch) (px/sec),Horizontal distance below WS (px) – BEFORE jump,Horizontal distance below WS (px) – AFTER jump,Max vertical distance below WS (px) – BEFORE jump,Max vertical distance below WS (px) – AFTER jump,AVG Underwater angle ( BEFORE jump),AVG Above water angle (during jump),AVG Underwater angle ( AFTER jump)
0,AC01,5.0,AC01_05,1.0,500.0,109.454659,75.0,173.0,232.584205,584.374943,231.855854,581.974672,50.660018,20.860298,20.0,83.0,3175.371269,4569.827467,5564.737709,55.206270,57.228400,70.0,81.0,11.0,185.0,217.0,32.0,12.0,2982.188438,1759.194011,599.838706,2216.507965,18.352077,52.768198,37.242273,62.447277,47.688230,50.088874,26.548743
1,AC01,6.0,AC01_06,2.0,500.0,92.387000,119.0,227.0,231.674277,661.324307,232.861424,660.450292,51.245615,16.680122,109.0,128.0,3415.465507,4274.838932,5471.713856,51.376215,58.727544,109.0,126.0,17.0,250.0,302.0,52.0,23.0,3062.332038,1059.470519,597.145285,1986.188473,23.097977,90.713124,38.431704,52.573140,45.566204,43.320038,25.425730
2,AC01,10.0,AC01_10,2.0,250.0,78.758538,150.0,197.0,163.056699,646.480562,161.508365,637.027535,30.504450,16.814773,134.0,157.0,3598.456519,2843.534630,4586.346968,38.316127,40.758666,148.0,155.0,7.0,224.0,237.0,13.0,27.0,3429.849436,1289.997757,789.360675,2179.407852,11.880640,127.558355,14.940107,30.579622,26.721839,32.949125,27.527812
4,AC03,4.0,AC03_04,1.0,250.0,102.045346,148.0,202.0,263.913803,712.212551,265.347982,709.444426,54.935321,21.11744,25.0,152.0,3464.488677,4599.310947,5758.154477,53.010654,56.242293,138.0,150.0,12.0,210.0,225.0,15.0,8.0,3300.388760,738.907714,523.381185,2213.533122,34.514278,53.295964,69.848653,55.487342,37.244899,53.747872,15.780241
5,AC03,15.0,AC03_15,1.0,500.0,105.265898,102.0,198.0,212.035651,625.119049,210.358030,621.853482,40.325637,21.432979,60.0,112.0,3740.098976,3921.153256,5418.835964,46.353789,46.139306,93.0,110.0,17.0,225.0,269.0,44.0,27.0,3258.247571,1228.981143,444.732420,1984.486270,24.076406,65.005577,34.372413,56.200804,37.602374,51.053788,25.132750
6,AC03,18.0,AC03_18,1.0,500.0,89.250502,100.0,190.0,157.288086,563.403060,154.307445,546.226143,41.330889,18.208605,36.0,107.0,3670.304940,3147.698165,4835.198247,40.616792,41.322035,91.0,105.0,14.0,211.0,240.0,29.0,21.0,3133.681016,1953.279988,429.292149,2077.438871,37.185894,46.898911,36.737300,45.347360,41.912005,43.390634,27.370503
7,AC03,20.0,AC03_20,1.0,500.0,110.340639,89.0,150.0,93.699816,447.732671,93.661617,443.923853,39.422690,18.050253,41.0,87.0,4276.140546,3962.674987,5829.937480,42.821095,40.336714,67.0,85.0,18.0,188.0,229.0,41.0,38.0,3685.137961,2650.615976,654.358423,1961.759669,120.467988,118.800418,123.495571,38.885787,36.013460,33.277995,19.950326
8,AC03,21.0,AC03_21,1.0,500.0,96.065096,212.0,297.0,145.146393,569.856531,144.942921,563.923406,27.234064,15.821628,174.0,226.0,3714.439000,2785.041747,4642.576290,36.862123,37.893723,204.0,223.0,19.0,416.0,460.0,44.0,119.0,3346.467398,928.141275,541.137183,1434.330135,14.595017,190.477295,14.610600,40.241873,21.808209,33.361854,14.248367
9,AC03,22.0,AC03_22,1.0,500.0,94.061775,260.0,344.0,133.591188,568.835927,133.863505,560.894386,27.544499,19.283003,229.0,273.0,3686.952559,2681.094884,4558.715712,36.024068,32.411075,245.0,271.0,26.0,463.0,509.0,46.0,119.0,3380.685225,625.843914,546.138704,1410

In [23]:
# There is np multiple jumps per sequence here, so we can go straight to interfrog

#for each frog in the list:
list_of_frogs = set(sheet['Frog'])

resmean ={}
resmin = {}
resmax = {}

for frog in list_of_frogs:
    #get subset of only that frog
    only_1frog = sheet[sheet['Frog'] == frog]
    ID = only_1frog.ID
    
    
    #and frames to time
    data_inframes = only_1frog.filter(regex='[F, f]rame')
    data_inframes.loc[:, 'wait_frames'] = only_1frog.loc[:, 'Wait time (btw prev land and prep start)']
    data_insec = data_inframes.div(only_1frog.fps, axis = 0)
    data_insec = data_insec.rename(columns=lambda x: re.sub(r'[F,f]rame(s)*', 'time(sec)', x ))
    #need to convert values to SVL
    data_inpx = only_1frog.filter(regex='px')
    data_incm = data_inpx.div(data_inpx['px/SVL'], axis= 0)
    data_incm = data_incm.mul(list(map(lambda z: frog_size[z], ID)), axis=0)
    data_incm = data_incm.rename(columns=lambda x: re.sub(r'px', 'cm', x ))
    #print(data_inSVL)
    data_angle = only_1frog.filter(regex='[A,a]ngle')
    
    #ok time to avg over frog:
    no_seq = len(only_1frog.index)
    mean_insec = data_insec.mean()
    mean_incm = data_incm.mean()
    mean_angle = data_angle.mean()
    totalmean = pd.concat([mean_insec, mean_incm, mean_angle, pd.Series(data={'numSeq':no_seq}, index=['numSeq'])])
    
    totalmin = pd.concat([data_insec.min(), data_incm.min(), data_angle.min(), pd.Series(data={'numSeq':no_seq}, index=['numSeq'])])
    totalmax = pd.concat([data_insec.max(), data_incm.max(), data_angle.max(), pd.Series(data={'numSeq':no_seq}, index=['numSeq'])])
    
    resmean[frog] = totalmean
    resmin[frog] = totalmin
    resmax[frog] = totalmax
    
strict_allmean = pd.DataFrame(resmean)
strict_allmin = pd.DataFrame(resmin)
strict_allmax = pd.DataFrame(resmax)
    
strict_allmean_weighted = DescrStatsW(strict_allmean.to_numpy().T, weights=strict_allmean.loc['numSeq'].to_numpy(), ddof=0)
strict_allmean['noWeightAVG'] = strict_allmean.mean(axis=1)  ##<------ FIX THIS - GOING TO USE ADDED ON COLUMNS TO CALCULATE THINGS
strict_allmean['noWeightSE'] = strict_allmean.sem(axis=1)
strict_allmean['noWeightSTD'] = strict_allmean.std(axis=1)
strict_allmean['noWeightVAR'] = strict_allmean.var(axis=1)
strict_allmean['weightedAVG_sm'] = strict_allmean_weighted.mean
strict_allmean['weightedSE_sm'] = strict_allmean_weighted.std_mean
strict_allmean['weightedSTD_sm'] = strict_allmean_weighted.std
strict_allmean['weightedVAR_sm'] = strict_allmean_weighted.var
strict_allmin['ALL'] = strict_allmin.min(axis=1)
strict_allmax['ALL'] = strict_allmax.max(axis=1)

    

/home/talcat/.virtualenvs/cricketfrog/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [24]:
strict_allmean

,AC01,AC03,AC04,noWeightAVG,noWeightSE,noWeightSTD,noWeightVAR,weightedAVG_sm,weightedSE_sm,weightedSTD_sm,weightedVAR_sm
time(sec) water exit,0.329333,0.329714,0.276000,0.311683,0.012616,0.135520,0.016949,0.324727,0.004873,0.015410,0.000237
time(sec) water land,0.529333,0.507143,0.376000,0.470825,0.033830,0.204026,0.038665,0.501273,0.012897,0.040784,0.001663
time(sec) of Max Velocity,0.350000,0.347429,0.300000,0.332476,0.011494,0.144922,0.019338,0.343818,0.004396,0.013902,0.000193
time(sec) start propulsion,0.316667,0.304286,0.260000,0.293651,0.012163,0.127636,0.015039,0.303636,0.004687,0.014822,0.000220
time(sec) End propulsion,0.344667,0.342857,0.300000,0.329175,0.010321,0.143712,0.018991,0.339455,0.003953,0.012502,0.000156
Propulsion Time (time(sec)),0.028000,0.038571,0.040000,0.035524,0.002676,0.015404,0.000220,0.035818,0.001519,0.004805,0.000023
time(sec) start prep (HL),0.588667,0.607429,0.532000,0.576032,0.016032,0.251974,0.058288,0.595455,0.006855,0.021676,0.000470
time(sec) end prep (HL),0.662000,0.687143,0.600000,0.649714,0.018311,0.284148,0.074135,0.672364,0.008027,0.025384,0.000644
Duration HL prep (time(sec)),0.073333,0.079714,0.068000,0.073683,0.002394,0.032150,0.000951,0.076909,0.001253,0.003964,0.000016
wait_time(sec),0.059333,0.100286,0.156000,0.105206,0.019809,0.051339,0.002334,0.094182,0.008380,0.026501,0.000702


In [344]:
clipbd.copy(strict_allmean.to_csv(sep=',', header='True'))

In [28]:
justmean = strict_allmean[['AC01', 'AC03', 'AC04']]

In [31]:
justmean.std (axis=1)

time(sec) water exit                                    0.030903
time(sec) water land                                    0.082867
time(sec) of Max Velocity                               0.028155
time(sec) start propulsion                              0.029793
time(sec) End propulsion                                0.025282
Propulsion Time (time(sec))                             0.006555
time(sec) start prep (HL)                               0.039270
time(sec) end prep (HL)                                 0.044852
Duration HL prep (time(sec))                            0.005865
wait_time(sec)                                          0.048521
cm/SVL                                                  0.110604
Actual height (cm)                                      1.798159
Actual range (cm)                                       2.457903
fit height (cm)                                         1.811939
fit range (cm)                                          2.703895
Vx @ Max Velocity (cm/sec

## Old Jump Definiton

In [422]:
sheet = pd.read_excel(datasheet, sheet_name='Individual Jump Data AUTOMATED')

In [423]:
# remove unnamed columsn:
sheet = sheet.loc[:, ~sheet.columns.str.contains('^Unnamed')]
#drop the spaces between frogs for easy readability
sheet= sheet[sheet['Frog'].notna()]

In [424]:
sheet

,Frog,Sequence,ID,Jmp #,fps,px/SVL,Frame start of jump (water exit),Frame water land,Amount of jump,Jump contains max height,Actual height (px),Actual range (px),fit height (px),fit range (px),Body Angle @ WC launch,Body Angle @ WC land,Idx of Max Velocity,Frame of Max Velocity,Vx @ Max Velocity (px/sec),Vy @ Max Velocity (px/sec),V @ Max Velocity (px/sec),V angle (trajectory angle) at Max V,Body Angle @ Max Velocity,Frame start propulsion,Frame End propulsion,Propulsion Time (frames),Frame start prep (HL),Frame end prep (HL),Duration HL prep (frames),Wait time (btw prev land and prep start),Avg hor Velocity above water (px/sec),Avg hor Velocity below water (px/sec) – after jump,Avg hor vel total (launch land launch) (px/sec),Horizontal distance below WS (px) – after jump,Max vertical distance below WS (px) – before jump,AVG Underwater angle ( before jump),AVG Above water angle (during jump)
0,AC01,4.0,AC01_04,1.0,500.0,94.219028,NaN,131.0,NaN,True,245.848349,NaN,245.273506,690.191350,NaN,21.004148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,401.121846,NaN,36.772680,NaN,NaN,NaN
1,AC01,4.0,AC01_04,2.0,500.0,94.219028,176.0,NaN,NaN,True,228.009527,NaN,228.177931,665.896669,21.004148,NaN,152.0,185.0,3399.719278,4131.829310,5350.710664,50.552038,53.759160,170.0,185.0,15.0,136.0,168.0,32.0,5.0,NaN,NaN,NaN,NaN,34.065308,29.665724,NaN
2,AC01,5.0,AC01_05,1.0,500.0,109.454659,75.0,173.0,NaN,True,232.584205,584.374943,231.855854,581.974672,42.348888,20.860298,20.0,83.0,3175.371269,4569.827467,5564.737709,55.206270,57.228400,70.0,81.0,11.0,NaN,NaN,NaN,NaN,2982.188438,757.954707,2192.000139,82.172997,55.555895,NaN,50.088874
3,AC01,5.0,AC01_05,2.0,500.0,109.454659,227.0,NaN,NaN,True,291.840773,NaN,290.596784,662.797267,20.860298,NaN,171.0,234.0,3192.739478,5071.767374,5993.030091,57.809084,61.706311,218.0,232.0,14.0,185.0,217.0,32.0,12.0,NaN,NaN,NaN,NaN,62.447277,29.892102,NaN
4,AC01,6.0,AC01_06,1.0,500.0,92.387000,NaN,42.0,NaN,False,NaN,NaN,NaN,NaN,NaN,17.901836,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,679.465425,NaN,104.106887,NaN,NaN,NaN
5,AC01,6.0,AC01_06,2.0,500.0,92.387000,119.0,227.0,NaN,True,231.674277,661.324307,232.861424,660.450292,17.901836,16.680122,109.0,128.0,3415.465507,4274.838932,5471.713856,51.376215,58.727544,109.0,126.0,17.0,84.0,107.0,23.0,42.0,3062.332038,678.612199,2012.507238,115.632623,45.109612,29.251941,43.320038
6,AC01,6.0,AC01_06,3.0,500.0,92.387000,312.0,NaN,NaN,False,NaN,NaN,NaN,NaN,16.680122,NaN,301.0,320.0,3265.925294,3542.645672,4818.361379,47.327388,52.706871,304.0,317.0,13.0,250.0,302.0,52.0,23.0,NaN,NaN,NaN,NaN,52.573140,27.285626,NaN
7,AC01,10.0,AC01_10,1.0,250.0,78.758538,NaN,33.0,NaN,False,NaN,NaN,NaN,NaN,NaN,17.492350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,626.784364,NaN,292.252474,NaN,NaN,NaN
8,AC01,10.0,AC01_10,2.0,250.0,78.758538,150.0,197.0,NaN,True,163.056699,646.480562,161.508365,637.027535,17.492350,16.814773,134.0,157.0,3598.456519,2843.534630,4586.346968,38.316127,40.758666,148.0,155.0,7.0,106.0,147.0,41.0,73.0,3429.849436,811.871963,2194.399393,135.146842,26.892169,14.774632,32.949125
9,AC01,10.0,AC01_10,3.0,250.0,78.758538,239.0,NaN,NaN,False,NaN,NaN,NaN,NaN,16.814773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,238.0,244.0,6.0,224.0,237.0,13.0,27.0,NaN,NaN,NaN,NaN,30.579622,27.391349,NaN


In [425]:
sheet.groupby(['Frog', 'ID'])['px/SVL'].count()

Frog  ID     
AC01  AC01_04    2
      AC01_05    2
      AC01_06    3
      AC01_10    3
AC03  AC03_04    2
      AC03_06    2
      AC03_09    1
      AC03_14    2
      AC03_15    2
      AC03_18    2
      AC03_20    2
      AC03_21    2
      AC03_22    2
      AC03_23    2
AC04  AC04_03    2
Name: px/SVL, dtype: int64

In [426]:
sheet.groupby(['Frog'])['px/SVL'].count()

Frog
AC01    10
AC03    19
AC04     2
Name: px/SVL, dtype: int64

In [427]:
by_frog = sheet.groupby('Frog')
by_ID = sheet.groupby('ID')
by_frogAndID = sheet.groupby(['Frog', 'ID'])

In [428]:
sheet.columns

Index(['Frog', 'Sequence', 'ID', 'Jmp #', 'fps', 'px/SVL',
       'Frame start of jump (water exit)', 'Frame water land',
       'Amount of jump', 'Jump contains max height', 'Actual height (px)',
       'Actual range (px)', 'fit height (px)', 'fit range (px)',
       'Body Angle @ WC launch', 'Body Angle @ WC land', 'Idx of Max Velocity',
       'Frame of Max Velocity', 'Vx @ Max Velocity (px/sec)',
       'Vy @ Max Velocity (px/sec)', 'V @ Max Velocity (px/sec)',
       'V angle (trajectory angle) at Max V', 'Body Angle @ Max Velocity',
       'Frame start propulsion', 'Frame End propulsion',
       'Propulsion Time (frames)', 'Frame start prep (HL)',
       'Frame end prep (HL)', 'Duration HL prep (frames)',
       'Wait time (btw prev land and prep start)',
       'Avg hor Velocity above water (px/sec)',
       'Avg hor Velocity below water (px/sec) – after jump',
       'Avg hor vel total (launch  land  launch) (px/sec)',
       'Horizontal distance below WS (px) – after jump',
  

In [429]:
from IPython.display import display
from collections import defaultdict

In [430]:
#for each frog:
by_frog = sheet.groupby('Frog')
by_ID = sheet.groupby('ID')
by_frogAndID = sheet.groupby(['Frog', 'ID'])
final_mean = defaultdict(dict)
final_min = defaultdict(dict)
final_max = defaultdict(dict)
final_tog = defaultdict(dict)
for frogID, data in by_frogAndID:
    #by sequence
    (frog, ID) = frogID
    SVLincm = frog_size[ID]
    #print(SVLincm)
#         #and frames to time
#     data_inframes = only_1frog.filter(regex='[F, f]rame')
#     data_inframes.loc[:, 'wait_frames'] = data_inframes.loc[:, 'Frame start prep (HL)'] - data_inframes.loc[:, 'Frame water exit']
#     data_insec = data_inframes.div(only_1frog.fps, axis = 0)
#     data_insec = data_insec.rename(columns=lambda x: re.sub(r'[F,f]rame(s)*', 'time(sec)', x ))
#     #need to convert values to SVL
#     data_inpx = only_1frog.filter(regex='px')
#     data_incm = data_inpx.div(data_inpx['px/SVL'], axis= 0)
#     data_incm = data_incm.mul(list(map(lambda z: frog_size[z], ID)), axis=0)
#     data_incm = data_incm.rename(columns=lambda x: re.sub(r'px', 'cm', x ))
#     #print(data_inSVL)
    data_inframes = data.filter(regex='[F,f]rame')
    #print(data_inframes)
    #print(data.fps)
    data_inframes.loc[:, 'wait_frames'] = data.loc[:,'Wait time (btw prev land and prep start)' ]
    data_insec = data_inframes.div(data.fps, axis=0)
    data_insec = data_insec.rename(columns=lambda x: re.sub(r'[F,f]rame(s)*', 'time(sec)', x ))
    #print(frog, ID)
    data_angle = data.filter(regex='[A,a]ngle')
    
    data_inpx = data.filter(regex='px')
    data_incm = data_inpx.div(data_inpx['px/SVL'], axis= 0)
    data_incm = data_incm.mul(frog_size[ID], axis=0)
    data_incm = data_incm.rename(columns=lambda x: re.sub(r'px', 'cm', x ))
       
    together = pd.concat([data_insec,data_incm, data_angle], axis=1)
    #display(frogID)
    #display(together)
    #together.loc['avg', :]= together.mean()
    #display(together)
    
    final_mean[frog][ID] = together.mean(skipna=True)
    final_min[frog][ID] = together.min()
    final_max[frog][ID] = together.max()
    final_tog[frog][ID] =  together
    #display(together.mean())
    #print('\n')
#meanbyseq = pd.DataFrame(final_mean, index = by_frogAndID.groups.keys())

#togbyseq = pd.concat(final_tog, axis=0).reset_index(level=0).rename({'level_0':'ID'}, axis=1)

#meanbyseq = meanbyseq.rename_axis(['Frog', 'ID'])

#togbyseq = togbyseq.rename_axis( 'ID', axis=0)

#meanbyseq=meanbyseq.reset_index(level=[0, 1]) #unheirachy shits

#togbyseq=togbyseq.reset_index(level=[0, 1]) #unheirachy shits

################ this puts it back into a multi-index form
a = pd.DataFrame.from_dict(dict(final_mean), orient='index').stack().to_frame()
meanbyseq = pd.DataFrame(a[0].values.tolist(), index=a.index).rename_axis(['Frog', 'ID'])

/home/talcat/.virtualenvs/py3-cv/lib/python3.5/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/talcat/.virtualenvs/py3-cv/lib/python3.5/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [431]:
clipbd.copy(pd.DataFrame.from_dict(meanbyseq).to_csv(sep=',', header=True))

In [432]:
clipbd.copy(meanbyseq.groupby('Frog').mean().T.to_csv(sep=',', header=True))

In [440]:
clipbd.copy((meanbyseq.groupby('Frog').mean()).std().to_csv(sep=',', header=True))

In [409]:
b = pd.DataFrame.from_dict(final_tog['AC01'], orient='index').stack().to_frame()
display(b)
display(b[0])
display(b[0][0].columns)
display(b.columns)
#pd.DataFrame(b[0].values.tolist(), index=b.index)

,,0
AC01_04,0,time(sec) start of jump (water exit) time(...
AC01_05,0,time(sec) start of jump (water exit) time(...
AC01_06,0,time(sec) start of jump (water exit) time(...
AC01_10,0,time(sec) start of jump (water exit) time(...


AC01_04  0       time(sec) start of jump (water exit)  time(...
AC01_05  0       time(sec) start of jump (water exit)  time(...
AC01_06  0       time(sec) start of jump (water exit)  time(...
AC01_10  0       time(sec) start of jump (water exit)  time(...
Name: 0, dtype: object

Index(['time(sec) start of jump (water exit)', 'time(sec) water land',
       'time(sec) of Max Velocity', 'time(sec) start propulsion',
       'time(sec) End propulsion', 'Propulsion Time (time(sec))',
       'time(sec) start prep (HL)', 'time(sec) end prep (HL)',
       'Duration HL prep (time(sec))', 'wait_time(sec)', 'cm/SVL',
       'Actual height (cm)', 'Actual range (cm)', 'fit height (cm)',
       'fit range (cm)', 'Vx @ Max Velocity (cm/sec)',
       'Vy @ Max Velocity (cm/sec)', 'V @ Max Velocity (cm/sec)',
       'Avg hor Velocity above water (cm/sec)',
       'Avg hor Velocity below water (cm/sec) – after jump',
       'Avg hor vel total (launch  land  launch) (cm/sec)',
       'Horizontal distance below WS (cm) – after jump',
       'Max vertical distance below WS (cm) – before jump',
       'Body Angle @ WC launch', 'Body Angle @ WC land',
       'V angle (trajectory angle) at Max V', 'Body Angle @ Max Velocity',
       'AVG Underwater angle ( before jump)',
       'AVG A

RangeIndex(start=0, stop=1, step=1)

In [390]:
a = pd.DataFrame.from_dict(dict(final_mean), orient='index').stack().to_frame()
#pd.DataFrame(a[0].values.tolist(), index=a.index)
display(a)

0
AC01 AC01_04  time(sec) start of jump (water exit)          ...
     AC01_05  time(sec) start of jump (water exit)          ...
     AC01_06  time(sec) start of jump (water exit)          ...
     AC01_10  time(sec) start of jump (water exit)          ...
AC03 AC03_21  time(sec) start of jump (water exit)          ...
     AC03_18  time(sec) start of jump (water exit)          ...
     AC03_04  time(sec) start of jump (water exit)          ...
     AC03_23  time(sec) start of jump (water exit)          ...
     AC03_06  time(sec) start of jump (water exit)          ...
     AC03_22  time(sec) start of jump (water exit)          ...
     AC03_09  time(sec) start of jump (water exit)          ...
     AC03_20  time(sec) start of jump (water exit)          ...
     AC03_14  time(sec) start of jump (water exit)          ...
     AC03_15  time(sec) start of jump (water exit)          ...
AC04 AC04_03  time(sec) start of jump (water exit)          ...

In [359]:
final_mean

defaultdict(dict,
            {'AC01': {'AC01_04': time(sec) start of jump (water exit)                    0.352000
              time(sec) water land                                    0.262000
              time(sec) of Max Velocity                               0.370000
              time(sec) start propulsion                              0.340000
              time(sec) End propulsion                                0.370000
              Propulsion Time (time(sec))                             0.030000
              time(sec) start prep (HL)                               0.272000
              time(sec) end prep (HL)                                 0.336000
              Duration HL prep (time(sec))                            0.064000
              wait_time(sec)                                          0.010000
              cm/SVL                                                  2.010000
              Actual height (cm)                                      5.054469
              A

In [347]:
meanbyseq[meanbyseq['ID']=='AC03_23']

,Frog,ID,time(sec) start of jump (water exit),time(sec) water land,time(sec) of Max Velocity,time(sec) start propulsion,time(sec) End propulsion,Propulsion Time (time(sec)),time(sec) start prep (HL),time(sec) end prep (HL),Duration HL prep (time(sec)),wait_time(sec),cm/SVL,Actual height (cm),Actual range (cm),fit height (cm),fit range (cm),Vx @ Max Velocity (cm/sec),Vy @ Max Velocity (cm/sec),V @ Max Velocity (cm/sec),Avg hor Velocity above water (cm/sec),Avg hor Velocity below water (cm/sec) – after jump,Avg hor vel total (launch land launch) (cm/sec),Horizontal distance below WS (cm) – after jump,Max vertical distance below WS (cm) – before jump,Body Angle @ WC launch,Body Angle @ WC land,V angle (trajectory angle) at Max V,Body Angle @ Max Velocity,AVG Underwater angle ( before jump),AVG Above water angle (during jump)
6,AC03,AC03_23,0.172,NaN,0.196,0.16,0.192,0.032,NaN,NaN,NaN,NaN,2.14,4.336547,NaN,4.311551,16.587744,92.538903,75.562669,119.470354,NaN,NaN,NaN,3.171331,NaN,28.361638,NaN,39.233367,41.303777,NaN,NaN


In [310]:
togbyseq[togbyseq['ID']=='AC03_23'].mean()

time(sec) start of jump (water exit)                    0.356000
time(sec) water land                                    0.384000
time(sec) of Max Velocity                               0.378000
time(sec) start propulsion                              0.342000
time(sec) End propulsion                                0.375000
Propulsion Time (time(sec))                             0.033000
time(sec) start prep (HL)                               0.406000
time(sec) end prep (HL)                                 0.496000
Duration HL prep (time(sec))                            0.090000
wait_time(sec)                                          0.022000
cm/SVL                                                  2.140000
Actual height (cm)                                      4.316317
Actual range (cm)                                      14.356976
fit height (cm)                                         4.294573
fit range (cm)                                         14.165773
Vx @ Max Velocity (cm/sec

In [311]:
togbyseq

[time(sec) start of jump (water exit)                    0.352000
 time(sec) water land                                    0.262000
 time(sec) of Max Velocity                               0.370000
 time(sec) start propulsion                              0.340000
 time(sec) End propulsion                                0.370000
 Propulsion Time (time(sec))                             0.030000
 time(sec) start prep (HL)                               0.272000
 time(sec) end prep (HL)                                 0.336000
 Duration HL prep (time(sec))                            0.064000
 wait_time(sec)                                          0.010000
 cm/SVL                                                  2.010000
 Actual height (cm)                                      5.054469
 Actual range (cm)                                            NaN
 fit height (cm)                                         5.050134
 fit range (cm)                                         14.464896
 Vx @ Max 

In [134]:
def toSecAndCm(frogID, data):
    #by sequence
    (frog, ID) = frogID
    SVLincm = frog_size[ID]
 
    data_inframes = data.filter(regex='[F,f]rame')
    #print(data_inframes)
    #print(data.fps)
    data_inframes.loc[:, 'wait_frames'] = data.loc[:,'Wait time (btw prev land and prep start)' ]
    data_insec = data_inframes.div(data.fps, axis=0)
    data_insec = data_insec.rename(columns=lambda x: re.sub(r'[F,f]rame(s)*', 'time(sec)', x ))
    print(frog, ID)
    
    data_inpx = data.filter(regex='px')
    data_incm = data_inpx.div(data_inpx['px/SVL'], axis= 0)
    data_incm = data_incm.mul(frog_size[ID], axis=0)
    data_incm = data_incm.rename(columns=lambda x: re.sub(r'px', 'cm', x ))
    
    data_angl = data.filter(regex='[A,a]ngle')
       
    together = pd.concat([data_insec,data_incm, data_angl], axis=1)
    #together.mean()
    
    #display(together)
    return together

In [138]:
by_frogAndID.groups.keys()

dict_keys([('AC03', 'AC03_18'), ('AC03', 'AC03_22'), ('AC03', 'AC03_09'), ('AC03', 'AC03_14'), ('AC01', 'AC01_05'), ('AC03', 'AC03_21'), ('AC03', 'AC03_23'), ('AC03', 'AC03_15'), ('AC04', 'AC04_03'), ('AC03', 'AC03_04'), ('AC03', 'AC03_06'), ('AC01', 'AC01_06'), ('AC01', 'AC01_04'), ('AC01', 'AC01_10'), ('AC03', 'AC03_20')])